In [112]:
#Import libraries
import numpy as np #handles data in vactorized manner
import pandas as pd #handles data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json#handles JSON files

from geopy.geocoders import Nominatim #converts an address into latitude and longitude values

import requests #handles requests
from pandas.io.json import json_normalize # transform JSON file into a pandas datafame

#Import matplotlib and plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Import k-means for clustering
from sklearn.cluster import KMeans

import folium #renders map

from math import radians, sin, cos, acos

In [216]:
# API Key Hidden

In [3]:
def lat_long_finder(row):
    try:
        geolocator = Nominatim(user_agent="tpt_explorer")
    #         address = 'Tiruchanur Road, Tirupati, Andhra Pradesh, IN'

        location = geolocator.geocode(row)
        latitude = location.latitude
        longitude = location.longitude

        return (latitude,longitude)
    except:
        return None,None

In [4]:
df = pd.read_csv("Tirupati_Wards_List.csv")
df.head()

,Location
0,"Akkarampalle, Tirupati, Andhra Pradesh, IN"
1,"Subbareddy Nagar, Tirupati, Andhra Pradesh, IN"
2,"TTD Quarters, Tirupati, Andhra Pradesh, IN"
3,"ASI Colony, Tirupati, Andhra Pradesh, IN"
4,"Sri Nagar, Tirupati, Andhra Pradesh, IN"


In [17]:
df['Lat_Long'] = df['Location'].apply(lat_long_finder)
df['latitude'] = df['Lat_Long'].apply(lambda x: x[0])
df['longitude'] = df['Lat_Long'].apply(lambda x: x[1])
df.dropna(axis=0, how='any',inplace=True)
df.drop(["Lat_Long"],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)
df.to_csv("Neighborhoods_Lat_Lng.csv",index=False)
df.head()

,Location,latitude,longitude
0,"TTD Quarters, Tirupati, Andhra Pradesh, IN",13.625078,79.423043
1,"Reddigunta, Tirupati, Andhra Pradesh, IN",13.758038,79.812770
2,"Tiruchanur Road, Tirupati, Andhra Pradesh, IN",13.631430,79.460467
3,"Renigunta Road, Tirupati, Andhra Pradesh, IN",13.638732,79.510476
4,"Karakambadi Road, Tirupati, Andhra Pradesh, IN",13.654813,79.469513


In [64]:
tpt_wards_df = pd.read_csv("Neighborhoods_Lat_Lng.csv")
kclusters = 20

df_kmeans = tpt_wards_df.drop(['Location'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_kmeans)

cluster_centers = [[cc[0], cc[1]] for cc in kmeans.cluster_centers_]
[x.append(cluster_centers.index(x)) for x in cluster_centers]
cluster_labels = [cc for cc in kmeans.labels_]
tpt_wards_df['Labels'] = pd.DataFrame([item for item in cluster_labels])

# create map
# 13.6432° N, 79.4360° E
map_clusters = folium.Map(location=[13.6432, 79.4360], zoom_start=12)
folium.Circle([lat, lon], radius=3000, color='white', fill=True, fill_opacity=0.25).add_to(map_clusters)



for lat, lon,Name in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='orange', fill=True, fill_opacity=0.25).add_to(map_clusters)
    
for lat, lon, Name in cluster_centers:
    folium.CircleMarker([lat, lon], radius=5,popup=Name, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_clusters)

       
map_clusters

In [101]:
def google_nearby_places(labels,latitudes, longitudes, radius=500):
    places_list = []
    search = "pharmacy"
    
    for label,latitude, longitude in zip(labels,latitudes, longitudes):          
        # create the API request URL
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&types={}&key={}'\
        .format(latitude, longitude,radius,search,google_api_key)

        response = requests.get(url).json()
        
        for i in range(len(response['results'])):
            places_list.append([(label,response['results'][i]['geometry']['location']['lat'],response['results'][i]['geometry']['location']['lng'],
                           response['results'][i]['id'],response['results'][i]['name'],response['results'][i]['types'][0],response['results'][i]['vicinity'])])
    
    return places_list

def places_nearby_df(places_list):
    nearby_places = pd.DataFrame([item for venues_list in places_list for item in venues_list])
    nearby_places.columns = ['Label','Latitude', 'Longitude', 'ID', 'Place Name', 'Place Type', 'Place Address']
    
    return nearby_places

In [102]:
cluster_centers_df = pd.DataFrame([lat_lon for lat_lon in cluster_centers])
cluster_centers_df.columns = ["latitude",'longitude','label']
cluster_centers_df.head()

,latitude,longitude,label
0,13.611688,79.424375,0
1,13.643783,79.435583,1
2,13.632251,79.420186,2
3,13.659750,79.476500,3
4,13.625100,79.385800,4


In [103]:
places_list = google_nearby_places(labels=cluster_centers_df['label'],latitudes=cluster_centers_df['latitude'],longitudes=cluster_centers_df['longitude'])
tpt_pharmacies_df = places_nearby_df(places_list)
print(tpt_pharmacies_df.shape)
tpt_pharmacies_df.head()

(113, 7)


,Label,Latitude,Longitude,ID,Place Name,Place Type,Place Address
0,1,13.643108,79.432346,b9ed4bda32716110d22af3e2a0a7edbf00eaca68,Sri Venkateswara Family Clinic,pharmacy,"1-47, Akkarampalli Road, Sriram Nagar, Tirupati"
1,2,13.633820,79.419643,3e0f71a8c3a9da22c9650844bdf532195a677b14,Apollo Pharmacy,pharmacy,"Door No, 114/B, Theertha Katta Street, Rayalas..."
2,2,13.634697,79.417964,add918ed9b94fa257081eab5ae597dd4d6e8b26d,Sai Balaji Medical & Surgical Distributors,pharmacy,"7-1-17 R N Mada Street, Tirupati"
3,2,13.631498,79.419658,8de32bfab1cbccc3ea7dffa95542a94559a80eaf,Apollo Pharmacy,pharmacy,"Door No 12/3, near Nalugu Kalla Mandapam, 328 ..."
4,2,13.631341,79.419634,b9f15662319fa73d2d57a87b2f75c3b440ca611e,Sri Galaxy Pharmacy,pharmacy,"Bandla Street, Mallaiah Gunta Katta, Tata Naga..."


In [109]:
pharma_df = pd.read_csv("Pharmacies.csv")
pharma_df.drop(["ID","Place Type"],1,inplace=True)
print(pharma_df.shape)
pharma_df.head()

(108, 5)


,Label,Latitude,Longitude,Place Name,Place Address
0,18,13.618047,79.433437,A1 Medical Shop,"Padmavathi Puram, Tirupati"
1,11,13.641823,79.427993,Amaravathi Medical & Fancy,"20-3-125/E Akkarampalli Road, Leelamahal, Junc..."
2,14,13.618670,79.409070,Anusha Medicals & Fancy,"D.No : 5-254, Ullipattida, Vasavi Nagar, M R P..."
3,2,13.633820,79.419643,Apollo Pharmacy,"Door No, 114/B, Theertha Katta Street, Rayalas..."
4,2,13.631498,79.419658,Apollo Pharmacy,"Door No 12/3, near Nalugu Kalla Mandapam, 328 ..."


In [170]:
pharma_df_centres = pharma_df.merge(cluster_centers_df ,how='inner', left_on="Label",right_on="label")
pharma_df_centres.drop(['label'],1,inplace=True)
pharma_df_centres.columns = ["Label","P_Latitude","P_Longitude","Place_Name","Place_Address","Centre_Latitude","Centre_Longitude"]
pharma_df_centres.head()

,Label,P_Latitude,P_Longitude,Place_Name,Place_Address,Centre_Latitude,Centre_Longitude
0,18,13.618047,79.433437,A1 Medical Shop,"Padmavathi Puram, Tirupati",13.62004,79.429923
1,18,13.618509,79.431161,Apollo Pharmacy,"Door No 4/65, Assesment No 276 Tiruchanur Road...",13.62004,79.429923
2,18,13.623181,79.427935,Apollo Pharmacy,"Door No 19/9/29/2A Sankaramadi Circle, Tirucha...",13.62004,79.429923
3,18,13.622570,79.430446,Bhargavi Medicals,"near Lakshmipuram Circle, Tiruchanoor Road, ne...",13.62004,79.429923
4,18,13.623437,79.429469,Jagannadham Medical Shop,"19-9-29A, Tiruchanoor Road, Kennedy Nagar, Tir...",13.62004,79.429923


In [171]:
def distance_calculator(P_Latitude,P_Longitude,Centre_Latitude,Centre_Longitude):
    slat = radians(float(P_Latitude))
    slon = radians(float(P_Longitude))
    elat = radians(float(Centre_Latitude))
    elon = radians(float(Centre_Longitude))

    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    
    return round(dist,2)

In [172]:
pharma_df_centres['Distance'] = pharma_df_centres.apply(lambda row: distance_calculator(row['P_Latitude'], row['P_Longitude'], row['Centre_Latitude'], row['Centre_Longitude']), axis=1)
pharma_df_centres.head()

,Label,P_Latitude,P_Longitude,Place_Name,Place_Address,Centre_Latitude,Centre_Longitude,Distance
0,18,13.618047,79.433437,A1 Medical Shop,"Padmavathi Puram, Tirupati",13.62004,79.429923,0.44
1,18,13.618509,79.431161,Apollo Pharmacy,"Door No 4/65, Assesment No 276 Tiruchanur Road...",13.62004,79.429923,0.22
2,18,13.623181,79.427935,Apollo Pharmacy,"Door No 19/9/29/2A Sankaramadi Circle, Tirucha...",13.62004,79.429923,0.41
3,18,13.622570,79.430446,Bhargavi Medicals,"near Lakshmipuram Circle, Tiruchanoor Road, ne...",13.62004,79.429923,0.29
4,18,13.623437,79.429469,Jagannadham Medical Shop,"19-9-29A, Tiruchanoor Road, Kennedy Nagar, Tir...",13.62004,79.429923,0.38


In [190]:
pharma_grouped = pd.DataFrame()
pharma_grouped['Label'] = cluster_centers_df['label']

pharma_grouped_dummy = pharma_df_centres[['Label','Distance']]
pharma_grouped_dummy = pharma_grouped_dummy.groupby(["Label"]).count().sort_values("Label", ascending=True)

pharma_dummy = pharma_grouped.merge(pharma_grouped_dummy,how='outer',on="Label")
pharma_dummy.fillna(0,inplace=True)
pharma_dummy.columns = ["Label","Count"]
pharma_dummy['Count'] = pharma_dummy['Count'].astype('int')
pharma_dummy.sort_values("Count", ascending=True)

,Label,Count
0,0,0
15,15,0
10,10,0
3,3,0
7,7,0
8,8,1
9,9,1
5,5,1
1,1,1
4,4,2


In [193]:
pharma_df_centres.head()

,Label,P_Latitude,P_Longitude,Place_Name,Place_Address,Centre_Latitude,Centre_Longitude,Distance
0,18,13.618047,79.433437,A1 Medical Shop,"Padmavathi Puram, Tirupati",13.62004,79.429923,0.44
1,18,13.618509,79.431161,Apollo Pharmacy,"Door No 4/65, Assesment No 276 Tiruchanur Road...",13.62004,79.429923,0.22
2,18,13.623181,79.427935,Apollo Pharmacy,"Door No 19/9/29/2A Sankaramadi Circle, Tirucha...",13.62004,79.429923,0.41
3,18,13.622570,79.430446,Bhargavi Medicals,"near Lakshmipuram Circle, Tiruchanoor Road, ne...",13.62004,79.429923,0.29
4,18,13.623437,79.429469,Jagannadham Medical Shop,"19-9-29A, Tiruchanoor Road, Kennedy Nagar, Tir...",13.62004,79.429923,0.38


In [215]:
tpt_wards_df = pd.read_csv("Neighborhoods_Lat_Lng.csv")
kclusters = 20

df_kmeans = tpt_wards_df.drop(['Location'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_kmeans)

cluster_centers = [[cc[0], cc[1]] for cc in kmeans.cluster_centers_]
[x.append(cluster_centers.index(x)) for x in cluster_centers]
cluster_labels = [cc for cc in kmeans.labels_]
tpt_wards_df['Labels'] = pd.DataFrame([item for item in cluster_labels])

# create map
map_clusters = folium.Map(location=[13.6288, 79.4192], zoom_start=12)
# folium.Circle([lat, lon], radius=3000, color='white', fill=True, fill_opacity=0.25).add_to(map_clusters)



for lat, lon,Name in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='orange', fill=True, fill_opacity=0.25).add_to(map_clusters)
    
for lat, lon,Name in zip(pharma_df_centres['P_Latitude'],pharma_df_centres['P_Longitude'],pharma_df_centres['Place_Name']):
    folium.CircleMarker([lat, lon], radius=3,popup=str(Name),  color='blue',fill=True, fill_color='lightgreen', fill_opacity=1).add_to(map_clusters)

    
       
map_clusters